In [1]:
import gluonbook as gb
from mxnet import nd
from mxnet.gluon import rnn

(corpus_indices, char_to_idx, idx_to_char,
 vocab_size) = gb.load_data_jay_lyrics()

D:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


以下部分对模型参数进行初始化。超参数`num_hiddens`定义了隐藏单元的个数。

In [2]:
# vocab_size = 1027
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = gb.try_gpu()

In [5]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)

    def _three():
        return (_one((num_inputs, num_hiddens)),
                _one((num_hiddens, num_hiddens)),
                nd.zeros(num_hiddens, ctx=ctx))

    W_xz, W_hz, b_z = _three()  # 更新门参数。
    W_xr, W_hr, b_r = _three()  # 重置门参数。
    W_xh, W_hh, b_h = _three()  # 候选隐藏状态参数。
    # 输出层参数。
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    # 创建梯度。
    params = [W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

定义模型
--------------
以下定义隐藏状态初始化函数`init_gru_state`。同“循环神经网络的从零开始实现”一节中定义的`init_rnn_state`函数一样，它返回由一个形状为（批量大小，隐藏单元个数）的值为 0 的`NDArray`组成的元组。

In [6]:
def init_gru_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

下面根据门控循环单元的计算表达式定义模型。

In [7]:
def gru(inputs, state, params):
    W_xz, W_hz, b_z, W_xr, W_hr, b_r, W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        Z = nd.sigmoid(nd.dot(X, W_xz) + nd.dot(H, W_hz) + b_z)
        R = nd.sigmoid(nd.dot(X, W_xr) + nd.dot(H, W_hr) + b_r)
        H_tilda = nd.tanh(nd.dot(X, W_xh) + R * nd.dot(H, W_hh) + b_h)
        H = Z * H + (1 - Z) * H_tilda
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

训练模型并创作歌词
----------------------
我们在训练模型时只使用相邻采样。设置好超参数后，我们将训练模型并根据前缀“分开”和“不分开”分别创作长度为 50 个字符的一段歌词。

In [8]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 160, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 40, 50, ['分开', '不分开']

我们每过 40 个迭代周期便根据当前训练的模型创作一段歌词。

In [10]:
gb.train_and_predict_rnn(gru, get_params, init_gru_state, num_hiddens,
                         vocab_size, ctx, corpus_indices, idx_to_char,
                         char_to_idx, False, num_epochs, num_steps, lr,
                         clipping_theta, batch_size, pred_period, pred_len,
                         prefixes)

epoch 40, perplexity 151.329829, time 3.32 sec
 - 分开 我想你你的让我不 你不你的让我不想 你不你的让我不想 你不你的让我不想 你不你的让我不想 你不你的
 - 不分开 我想你你的让我不想 你不你的让我不想 你不你的让我不想 你不你的让我不想 你不你的让我不想 你不你
epoch 80, perplexity 33.382917, time 3.43 sec
 - 分开 我想要这样 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
 - 不分开 我想要你想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我不要再想 我
epoch 120, perplexity 5.174575, time 3.25 sec
 - 分开我 一场走 一颗四三 三头四 的片段 有一些好霜 老人盘 的片段 有一些好霜 老人在 旧皮了空屋 白
 - 不分开 你爱我 你是是一个人 我的完头 我给一定节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该
epoch 160, perplexity 1.461390, time 3.43 sec
 - 分开 一直到酒落忧 它在许里 是谁在练太极 风生水起 快使用双截棍 哼哼哈兮 快使用双截棍 哼哼哈兮 快
 - 不分开 爱过我依已经痛 我叫你的微笑笑 想通 你爱很久倒我 说散 你想很久了吧? 败给你的黑色幽默 说散 


Gluon 实现
-------------
在 Gluon 中我们直接调用`rnn`模块中的`GRU`类即可。

Gluon的实现链接为：https://zh.gluon.ai/chapter_recurrent-neural-networks/gru.html

In [ ]:
gru_layer = rnn.GRU(num_hiddens)
model = gb.RNNModel(gru_layer, vocab_size)
gb.train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                              corpus_indices, idx_to_char, char_to_idx,
                              num_epochs, num_steps, lr, clipping_theta,
                              batch_size, pred_period, pre_len, prefixes)